In [1]:
import glob
import re
import pandas as pd

#### Starter code for open files

In [8]:
xPositive = []
xNegative = []
xSeq = []
yContinous = []
yDiscrete = []
replaceWithNoSpace = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
replaceWithSpace = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

for s in ['pos', 'neg']: 
    files = glob.glob('./comp-551-imbd-sentiment-classification/train/'+ s +'/*.txt')
    files.sort(key = lambda x: int(x.split('/')[-1].split('_')[0]))
    for name in files:
        yContinous.append(int(name.split('_')[1].replace('.txt', '')))
        xSeq.append(int(name.split('/')[-1].split('_')[0]))
        if s == 'pos': 
            yDiscrete.append(1)
        else: 
            yDiscrete.append(0)
        with open(name) as f:
            txt = f.read()
            txt2 = replaceWithNoSpace.sub('', txt.lower())
            txtf = replaceWithSpace.sub(' ', txt2)
            if s == 'pos': 
                xPositive.append(txtf)
            else: 
                xNegative.append(txtf)

In [9]:
df = pd.DataFrame([[x, yd, yc] for x, yd, yc in zip(xPositive + xNegative, yDiscrete, yContinous)], columns=['text', 'y discrete', 'y continous'])

#### split the text into training sets and test set

In [10]:
fold = 9
totalN = df.shape[0]
posData = df[0:int(totalN/2)]
negData = df[int(totalN/2): totalN]
trainPos, testPos = posData[0:11250], posData[11250:12500]
trainNeg, testNeg = negData[0:11250], negData[11250:12500]

#### Divide the training sets into 9 subsets. Everytime choose one portion as validation set and the others as training sets

In [11]:
from sklearn.model_selection import KFold
from numpy import matrix
import numpy as np

# the output will be a list of size 9
trainPosMat = matrix(trainPos)
trainNegMat = matrix(trainNeg)
kf = KFold(n_splits = fold)
kf.get_n_splits(trainPos)
print(trainPos)
# iteratively produce the train set and the validation set
for train_index, validation_index in kf.split(trainPosMat):
    trainP, validationP = trainPosMat[train_index], trainPosMat[validation_index]
    trainN, validationN = trainNegMat[train_index], trainNegMat[validation_index]
    trainBoth = np.concatenate((trainP, trainN))
    validationBoth = np.concatenate((validationP, validationN))
    # call the model

                                                    text  y discrete  \
0      bromwell high is a cartoon comedy it ran at th...           1   
1      if you like adult comedy cartoons like south p...           1   
2      bromwell high is nothing short of brilliant ex...           1   
3      all the worlds a stage and its people actors i...           1   
4      futz is the only show preserved from the exper...           1   
5      i came in in the middle of this film so i had ...           1   
6      fair drama love story movie that focuses on th...           1   
7      although i didnt like stanley & iris tremendou...           1   
8      very good drama although it appeared to have a...           1   
9      working class romantic drama from director mar...           1   
10     im a male not given to womens movies but this ...           1   
11     liked stanley & iris very much acting was very...           1   
12     liked stanley & iris very much acting was very...        

#### Features for bernoulli Naive Bayes

In [12]:
#give the adjectives appearing in the data
import nltk
def findAdjectiveWords(data, n):
    counter = dict()
    for item in data['text']:
        word = item.lower().split()
        for each, tag in nltk.pos_tag(word):
            if tag=="JJ" or tag=="JJR" or tag=="JJS":
                if each in counter:
                    counter[each]+=1
                else:
                    counter[each]=1
    
    topWordsAndCount = [(k, counter[k]) for k in sorted(counter, key=counter.get, reverse=True)]
    topWords = [word[0] for word in topWordsAndCount]
    topnWords = topWords[0:n]
             
    print(topnWords)
    return topnWords

goodWord = findAdjectiveWords(trainPos, 150)
badWord = findAdjectiveWords(trainNeg, 150)

['good', 'i', 'great', 'other', 'best', 'many', 'first', 'more', 'little', 'much', 'real', 'new', 'young', 'old', 'such', 'most', 'same', 'own', 'few', 'bad', 'big', 'funny', 'excellent', 'last', 'different', 'better', 'true', 'beautiful', 'original', 'wonderful', 'only', 'american', 'whole', 'ive', 'hard', 'least', 'nice', 'high', 'perfect', 'interesting', 'main', 'classic', 'long', 'small', 'special', 'sure', 'black', 'human', 'full', 'short', 'second', 'favorite', 'early', 'top', 'fine', 'right', 'several', 'brilliant', 'next', 'white', 'strong', 'final', 'able', 'dead', 'overall', 'fantastic', 'simple', 'wrong', 'amazing', 'hilarious', 'late', 'entire', 'happy', 'musical', 'low', 'enjoyable', 'worth', 'romantic', 'important', 'modern', 'usual', 'greatest', 'british', 'less', 's', 'easy', 'realistic', 'serious', 'entertaining', 'similar', 'enough', 'dark', 'powerful', 'english', 'im', 'comic', 'french', 'due', 'memorable', 'local', 'emotional', 'dont', 'certain', 'unique', 'famous',

In [13]:
allWords = list(set(goodWord).union(set(badWord)))
print(allWords)

['dead', 'same', 'japanese', 'black', 'able', 'old', 'boring', 'laughable', 'rich', 'true', 'top', 'usual', 'difficult', 'positive', 'favorite', 'lame', 'ok', 'simple', 'such', 'red', 'worst', 'better', 'beautiful', 'dramatic', 'easy', 'amazing', 'nice', 'general', 'female', 'sure', 'big', 'wrong', 'hilarious', 'huge', 'hot', 'particular', 'many', 'poor', 's', 'political', 'potential', 'classic', 'fantastic', 'comic', 'last', 'decent', 'pathetic', 'enjoyable', 'several', 'french', 'brilliant', 'perfect', 'right', 'effective', 'half', 'original', 'great', 'long', 'stupid', 'weak', 'obvious', 'disappointed', 'scary', 'typical', 'cool', 'certain', 'unique', 'bad', 'personal', 'serious', 'previous', 'final', 'pretty', 'awful', 'outstanding', 'third', 'complete', 'wonderful', 'flat', 'own', 'important', 'terrible', 'strange', 'least', 'various', 'only', 'romantic', 'happy', 'weird', 'high', 'free', 'different', 'predictable', 'i', 'sad', 'im', 'ive', 'overall', 'interesting', 'strong', 'pos

#### Construct the features as binary, check if the word is present or not

In [14]:
trainAll = pd.concat([trainPos[0:10000], trainNeg[0:10000]])
validationAll = pd.concat([trainPos[10000:11250], trainNeg[10000:11250]])

def featuresConstruction(data, allWords):
    adjFeatures=[]
    for text in data['text']:
        adjFeature = dict()
        for word in allWords:
            if word in text:
                adjFeature[word]=1
            else:
                adjFeature[word]=0
        temp = [adjFeature[k] for k in adjFeature]
        adjFeatures.append(temp)

    adjFeatures = np.array(adjFeatures)
    return adjFeatures

adjFeatures = featuresConstruction(trainAll, allWords)
validationFeatures = featuresConstruction(validationAll, allWords)

In [15]:
yDiscrete = np.atleast_2d(trainAll["y discrete"]).T
yValidation = validationAll["y discrete"]
yValidation = np.array(yValidation)

### Fit function

In [16]:
# the function takes in the feature matrix and the response matrix
def BernoulliNaiveBayes(X, y):
    dataSize = y.size
    classCount = dict()
    classCount["1"] = sum(y) # count of y=1
    classCount["0"] = dataSize - classCount["1"] # count of y=0
    priorProb = {k: v / dataSize for k, v in classCount.items()} # P(y=1) and P(y=0)

    XandY=np.concatenate((X,y), axis = 1)
    
    
    eachTotalCount = sum(X) #the total number of counts for each word
    positiveRow = []
    negativeRow = []
    
    for row in XandY:
        if row[XandY.shape[1]-1]==1:
            positiveRow.append(row)
        else:
            negativeRow.append(row)
    positiveRow = np.delete(np.array(positiveRow), -1, axis=1)

    
    # calculate the counts of words in positive set and negative set
    presentPosCount = sum(positiveRow) #number of instance with xj=1 and y=1
    presentNegCount = eachTotalCount - presentPosCount #number of instance with xj=1 and y=0
    
    # the probability after laplace smoothing
    presentPosProb = (presentPosCount+1)/(classCount["1"]+2) #P(xj=1|y=1)
    presentNegProb = (presentNegCount+1)/(classCount["0"]+2) #P(xj=1|y=0)
    absentPosProb = 1-presentPosProb #P(xj=0|y=1)
    absentNegProb = 1-presentNegProb #P(xj=0|y=0)
    
    w_j0 = np.log(np.divide(absentPosProb, absentNegProb))
    w_j1 = np.log(np.divide(presentPosProb, presentNegProb))  
    
    
    w0 = np.log(np.divide(priorProb["1"], priorProb["0"]))+sum(w_j0)
    w = w_j1 - w_j0
    
    return w0, w


w0, w = BernoulliNaiveBayes(adjFeatures, yDiscrete)
w.shape

(185,)

### Prediction function

In [20]:
def predict(w0, w, validationX):
    prediction = []
    for row in validationX:
        result = w0+np.dot(row.T, w)
        if result>0:
            prediction.append("1")
        else:
            prediction.append("0")
    return prediction 

In [57]:
prediction = predict(w0, w, validationFeatures)
prediction = np.array(prediction)
prediction.shape
prediction.shape
yValidation.shape

(2500,)

In [61]:
from sklearn.metrics import f1_score
prediction = [int(x) for x in prediction]
f1_score(yValidation,prediction)

0.810875649740104